С помощью скрипта можно относительно быстро получить список отзывов к определенному объекту на Google.Карты.

Для начала работы нам необходимо подгрузить модули для работы с сохраненной страницей в формате html.

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re

Далее нам нужно загрузить наш файл с сохраненной страницей в формате html, изменив путь в строке. Для этого в Colab найдите слева иконку папки и найдите кнопку загрузки файла (иконка под словом "Файлы"). Загрузив страницу, нажмите по загруженному файлу правой кнопкой мыши и выберите "Скопировать путь". В строке с кодом внизу замените запись на ту, что скопировали, предварительно удалив содержимое внутри скобок в кавычках.

In [ ]:
with open('/content/Музей Н. А. Островского – Google Карты.html') as f:
    content = f.read()

soup = BeautifulSoup(content, features='html.parser')

Функция для извлечения комментариев для Google.Карт

In [ ]:
def get_reviews_from_GoogleMaps(bs4_soup = soup):
    reviews = []
    authors = []
    authors_status = []
    authors_ratings = []
    publication_date = []

    for result in soup.findAll('div', class_="jftiEf fontBodyMedium"):
        try:
            reviews.append(result.find('span', class_='wiI7pd').get_text())
        except:
            reviews.append(np.NaN)

        try:
            authors.append(result.find('div', class_='d4r55').find('span').\
                           get_text())
        except:
            authors.append(np.NaN)
            
        try:
            authors_status.append(result.find('div', class_='RfnDt').\
                                  find('span').get_text())
        except:
            authors_status.append(np.NaN)
        
        try:
            authors_ratings.append(result.\
                                   find('span', class_='kvMYJc')['aria-label'])
        except:
            authors_ratings.append(None)
        
        try:
            publication_date.append(result.find('span', class_='rsqaWe').get_text())
        except:
            publication_date.append(None)

    authors_ratings_int = [int(re.findall(r'\d+', n)[0]) for n in authors_ratings]

    data = pd.DataFrame.from_dict({'author': authors,\
                               'status': authors_status,\
                               'rating': authors_ratings_int,\
                               'publication_date': publication_date,\
                               'review': reviews})

    return data


Переменная с обработанным функцией результатом парсинга страницы.

In [ ]:
data = get_reviews_from_GoogleMaps(bs4_soup = soup)

In [ ]:
data.head(10) # вывод первых десяти строк таблицы

Скрипт сохраняет данные в формате .xlsx в раздел "Файлы", откуда затем файлы можно скачать к себе на рабочий стол.

In [ ]:
data.to_excel('result.xlsx')